In [1]:
#'##########################################################
#	  	      MCMC GLMM
# 	a sript for  the project
#	DEMOGRAPHIC BUFFERING CONTINUUM in PLANTS AND ANIMALS
#			 by Gabriel Santos
# 	contact by ssantos.gabriel@gmail.com
#			    05 March 2025
#' ---------------------------------------------------------
# MCMCglmms run separately so we can run it
# with intermediary data produced
#'##########################################################
set.seed(1)

rm(list=ls())

# Define necessary packages
need_pkgs <- c("tidyverse", "plotMCMC", "mcmcr", "MuMIn", "MCMCglmm","MuMIn")

exist_pckgs<-exist_pckgs <- installed.packages()[, "Package"]

if (any(!need_pkgs %in% exist_pckgs)) {   # Check for inexisting packages and install them
  install.packages(need_pkgs[!need_pkgs %in% exist_pckgs])
}




In [2]:
# load non-existing packages
lapply(need_pkgs, require, character.only = TRUE)

rm(list=ls())

Loading required package: tidyverse
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the ]8;;http://conflicted.r-lib.org/conflicted package]8;; to force all conflicts to become errors
Loading required package: plotMCMC
Loading required package: mcmcr
Registered S3 method overwritten by 'mcmcr':
  method         from 
  as.mcmc.nlists nlist
Loading required package: MuMIn
Loading required package: MCMCglmm
Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack

Loading required package: coda
Loading required package: ape

A

Warning messages:
1: package ‘plotMCMC’ was built under R version 4.4.3 
2: package ‘mcmcr’ was built under R version 4.4.3 
3: package ‘MCMCglmm’ was built under R version 4.4.2 


In [3]:

#'===========================================================
# ----- LOAD data ready for GLMM analyses ----
# GLMM data contains:
#  - Data model
#  - subtree_Animals - Phylogenetic tree for Animals ready for analysis
#  - subtree_Animals - Phylogenetic tree for Plants ready for analysis
#'===========================================================
# USE THIS IF RUNNING IN GOOGLE COLAB
# Download data In Google colab
#dir.create(file.path("Data"), showWarnings = FALSE) # Will return warning message if folder already exists
#GLMMdata_link<-"https://github.com/Ecosantos/Demogbuff-pops/raw/refs/heads/incorporating-MCMCGlmm/Data/GLMMdata.Rdata"
#download.file(GLMMdata_link, "Data/GLMMdata.Rdata", mode = "wb")
#'===========================================================

load("Data/GLMMdata.Rdata")

data_model%>%glimpse()
data_model$Reproduction_SigElas<-abs(data_model$Reproduction_SigElas)
data_model$Growth_SigElas<-abs(data_model$Growth_SigElas)
data_model$Shrinking_SigElas<-abs(data_model$Shrinking_SigElas)
data_model$Clonality_SigElas<-abs(data_model$Clonality_SigElas)
data_model$Survival_SigElas<-abs(data_model$Survival_SigElas)

#'===========================================================
# ---- Define GLMM SETTINGS ----
#'===========================================================

fixEffect<-fixEffect<-"~LHPC.1 * LHPC.2 + ClimPC.1 * ClimPC.2"
InterestingVars<-c("Survival","Growth","Shrinking","Reproduction","Clonality","Buffmx","Cumulative")

traits<-traits_glmm<- unique (grep(paste(InterestingVars,collapse="|"),
                                   colnames(data_model), value=TRUE))

#'-----------------------------------------------------------
# ---- Define priors ----
#'-----------------------------------------------------------

prior_phylo<-list(G=list(G1=list(V=1,nu=0.02)),
                  R=list(V=1,nu=0.02))

nitt=100000; #nitt=1000
burnin=1000; #burnin=100
thin=10

glmmScale<-"FALSE"

Rows: 134
Columns: 25
$ ID                   <chr> "Urs2.183_726", "Flcn.170_257", "Lgps.394_383", "…
$ Reproduction_SigElas <dbl> 1.364528e-04, -1.938652e-03, -2.377753e-02, -1.02…
$ Growth_SigElas       <dbl> 7.525562e-04, 0.000000e+00, 0.000000e+00, 0.00000…
$ Shrinking_SigElas    <dbl> 4.928246e-04, 0.000000e+00, 0.000000e+00, 0.00000…
$ Clonality_SigElas    <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ Survival_SigElas     <dbl> -1.692481e-02, -7.611464e-03, -8.279373e-02, -9.8…
$ Cumulative_SigElas   <dbl> 0.0155429721, 0.0095501157, 0.1065712526, 0.10840…
$ Buffmx               <dbl> 1.0205700, 1.0084916, 1.1488676, 1.1131551, 1.017…
$ MatRep               <int> 5, 5, 5, 6, 5, 3, 3, 4, 9, 11, 3, 3, 4, 4, 53, 4,…
$ LHPC.1               <dbl> 4.0555225, -2.9624200, -3.5976811, 1.6025393, 1.0…
$ LHPC.2               <dbl> -0.69666310, -0.12995466, 0.73974502, -0.61097725…
$ ClimPC.1             <dbl> 3.3882768, -0.2612499, 0.1603933, 0.1603933, -0.5…
$ ClimPC.2        

In [4]:
#'===========================================================
#	PHYLOGENETIC MCMC GLMM
#'===========================================================
MCMCglmm_phylo_animals<-MCMCglmm_phylo_plants<-NULL

#Animals	phylo
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_phylo_animals[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
    random=~phylo,family="gaussian",
		ginverse=list(phylo=inverseA(subtree_Animals,nodes="TIPS",scale=TRUE)$Ainv),
				prior=prior_phylo,data=subset(data_model,Kingdom=="Animalia"),
   						nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_phylo_animals)[i]<-traits[[i]]
}


#Plants
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_phylo_plants[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
    random=~phylo,family="gaussian",
		ginverse=list(phylo=inverseA(subtree_Plants,nodes="TIPS",scale=TRUE)$Ainv),
				prior=prior_phylo,data=subset(data_model,Kingdom=="Plantae"),
   						nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_phylo_plants)[i]<-traits[[i]]
}

#'===========================================================
#	SIMPLE MCMC GLMM
#'===========================================================
MCMCglmm_simple_animals<-MCMCglmm_simple_plants<-NULL

#Animals
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_simple_animals[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
    family="gaussian",data=subset(data_model,Kingdom=="Animalia"),
   			nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_simple_animals)[i]<-traits[[i]]
}

#Plants
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_simple_plants[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
   family="gaussian",data=subset(data_model,Kingdom=="Plantae"),
   			nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_simple_plants)[i]<-traits[[i]]
}







[1] "Running model: Reproduction_SigElas ~ ~LHPC.1 * LHPC.2 + ClimPC.1 * ClimPC.2"

                       MCMC iteration = 0

                       MCMC iteration = 1000

                       MCMC iteration = 2000

                       MCMC iteration = 3000

                       MCMC iteration = 4000

                       MCMC iteration = 5000

                       MCMC iteration = 6000

                       MCMC iteration = 7000

                       MCMC iteration = 8000

                       MCMC iteration = 9000

                       MCMC iteration = 10000

                       MCMC iteration = 11000

                       MCMC iteration = 12000

                       MCMC iteration = 13000

                       MCMC iteration = 14000

                       MCMC iteration = 15000

                       MCMC iteration = 16000

                       MCMC iteration = 17000

                       MCMC iteration = 18000

                       MCMC iteratio

In [12]:
glmmOUT<-list(Simple_models=
       list(Plants = MCMCglmm_simple_plants,Animals = MCMCglmm_simple_animals),
     Phylogenetic_models=
       list(Plants = MCMCglmm_phylo_plants,Animals = MCMCglmm_phylo_animals)
     )

saveRDS(glmmOUT,"Data/MCMCglmm_output.rds")

In [7]:
Phylo_models_plants<-lapply(MCMCglmm_phylo_plants,function(inner_list) summary(inner_list))

Simple_models_plants<-lapply(MCMCglmm_simple_plants,function(inner_list) summary(inner_list))


Phylo_models_plants_coefs<-lapply(Phylo_models_plants,function(inner_list) inner_list[[5]])
Simple_models_plants_coefs<-lapply(Simple_models_plants,function(inner_list) inner_list[[5]])

Phylo_models_plants_coefs<-lapply(Phylo_models_plants_coefs,as.data.frame)
Simple_models_plants_coefs<-lapply(Simple_models_plants_coefs,as.data.frame)


Phylo_models_df_plants<-lapply(Phylo_models_plants_coefs,rownames_to_column, var = "Statistics")%>%
  Map(cbind, Trait = names(.),Taxa="Plants", Model="Phylo", .)%>%do.call(rbind,.)

Simple_models_df_plants<-lapply(Simple_models_plants_coefs,rownames_to_column, var = "Statistics")%>%
  Map(cbind, Trait = names(.),Taxa="Plants", Model="Simple", .)%>%do.call(rbind,.)


GLMMs_df<-rbind(Phylo_models_df_plants,
                Simple_models_df_plants)


colnames(GLMMs_df)<-c("Trait","Taxa","Model","Statistics","post.mean","low95","high95","eff.samp","pMCMC")


#--------------------------------------------------------------------
# -----  SUMMARY SINTHESIS  -----  
#--------------------------------------------------------------------

GLMMs_df_summary<-GLMMs_df%>%
  mutate(sig=ifelse(pMCMC<=0.05,"Sig","Non-Sig"))%>%
  filter(Statistics!="(Intercept)")

In [8]:
GLMMs_df_summary

                                       Trait   Taxa  Model        Statistics
Reproduction_SigElas.2  Reproduction_SigElas Plants  Phylo            LHPC.1
Reproduction_SigElas.3  Reproduction_SigElas Plants  Phylo            LHPC.2
Reproduction_SigElas.4  Reproduction_SigElas Plants  Phylo          ClimPC.1
Reproduction_SigElas.5  Reproduction_SigElas Plants  Phylo          ClimPC.2
Reproduction_SigElas.6  Reproduction_SigElas Plants  Phylo     LHPC.1:LHPC.2
Reproduction_SigElas.7  Reproduction_SigElas Plants  Phylo ClimPC.1:ClimPC.2
Growth_SigElas.2              Growth_SigElas Plants  Phylo            LHPC.1
Growth_SigElas.3              Growth_SigElas Plants  Phylo            LHPC.2
Growth_SigElas.4              Growth_SigElas Plants  Phylo          ClimPC.1
Growth_SigElas.5              Growth_SigElas Plants  Phylo          ClimPC.2
Growth_SigElas.6              Growth_SigElas Plants  Phylo     LHPC.1:LHPC.2
Growth_SigElas.7              Growth_SigElas Plants  Phylo ClimPC.1:ClimPC.2

In [11]:
GLMMs_df_summary%>%
  filter(Taxa=="Plants" & !c(Trait %in% c("Cumulative_SigElas","Buffmx")))%>%
  group_by(Trait)%>%group_split()

<list_of<
  tbl_df<
    Trait     : character
    Taxa      : character
    Model     : character
    Statistics: character
    post.mean : double
    low95     : double
    high95    : double
    eff.samp  : double
    pMCMC     : double
    sig       : character
  >
>[5]>
[[1]]
# A tibble: 12 × 10
   Trait  Taxa  Model Statistics post.mean    low95  high95 eff.samp pMCMC sig  
   <chr>  <chr> <chr> <chr>          <dbl>    <dbl>   <dbl>    <dbl> <dbl> <chr>
 1 Clona… Plan… Phylo LHPC.1     -0.00261  -0.00896 0.00331    9565. 0.394 Non-…
 2 Clona… Plan… Phylo LHPC.2     -0.000916 -0.0116  0.00913    9900. 0.863 Non-…
 3 Clona… Plan… Phylo ClimPC.1    0.00158  -0.0104  0.0125    10809. 0.782 Non-…
 4 Clona… Plan… Phylo ClimPC.2   -0.00398  -0.0160  0.00769    9900. 0.505 Non-…
 5 Clona… Plan… Phylo LHPC.1:LH…  0.00120  -0.00427 0.00660    9457. 0.670 Non-…
 6 Clona… Plan… Phylo ClimPC.1:… -0.00174  -0.00935 0.00576   10363. 0.644 Non-…
 7 Clona… Plan… Simp… LHPC.1     -0.000518 -0.00723